# Dataset Scraping

## Imports

In [228]:
from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn
import requests
import shutil
import os
import time
import math
import pandas
import json
import zipfile
from tqdm import tqdm
from glob import glob
import multiprocessing as mp


## Globals

In [401]:
timestr = time.strftime("%Y%m%d-%H%M%S")

PERCENT_TO_SCRAPE = 0.00015
NUMBER_TO_SKIP = 40000
DATASET_PATH = "/projectnb/sparkgrp/ml-herbarium-data/"
DATASET_ARCHIVE = "data.zip"
DATASET_CSV = "data.csv"
OUTPUT_PATH = "/projectnb/sparkgrp/ml-herbarium-data/scraped-data/" + timestr + "/"
DATASET_URL = "https://occurrence-download.gbif.org/occurrence/download/request/0196625-210914110416597.zip"
DATASET_TYPE = ""
NUM_CORES = 50


## Download Dataset
#### Only run this if the dataset needs to be redownladed

In [209]:
if os.path.exists(DATASET_PATH + DATASET_ARCHIVE):
    os.remove(DATASET_PATH + DATASET_ARCHIVE)
ds = requests.get(DATASET_URL, stream=True)
total_size_in_bytes = int(ds.headers.get("content-length", 0))
block_size = 1024  # 1 Kibibyte
progress_bar = tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)
with open(DATASET_PATH + DATASET_ARCHIVE, "wb") as f:
    for data in ds.iter_content(block_size):
        progress_bar.update(len(data))
        f.write(data)
progress_bar.close()
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    print("ERROR, something went wrong")


 10%|▉         | 113M/1.16G [00:20<03:10, 5.52MiB/s]
100%|██████████| 1.16G/1.16G [01:48<00:00, 10.7MiB/s]


## For DWCA files

### Open DWCA File

In [192]:
dwca = DwCAReader(DATASET_PATH + DATASET_ARCHIVE)
DATASET_TYPE = "dwca"


### Test DWCA
Will throw an error if the file is not opened correctly.

In [6]:
dwca.get_corerow_by_position(0)


### Save DWCA Rows to Pandas Dataframe

In [7]:
# df = dwca.pd_read('occurrence.txt')
df = dwca.pd_read("occurrence.txt", low_memory=False)


#### Close the archive to free resources


In [8]:
dwca.close()


## For CSV files

### Extract CSV from zip file

In [ ]:
if os.path.exists(DATASET_PATH + "*.csv"):
    os.remove(DATASET_PATH + "*.csv")
with zipfile.ZipFile(DATASET_PATH + DATASET_ARCHIVE, "r") as zip_ref:
    zip_ref.extractall(DATASET_PATH)
f = glob(os.path.join(DATASET_PATH, "*-*.csv"))[0]
os.rename(f, DATASET_PATH + DATASET_CSV)
print("CSV file extracted")


### Save CSV Rows to Pandas Dataframe

In [217]:
df = pandas.read_csv(DATASET_PATH + DATASET_CSV, sep="\t")
DATASET_TYPE = "csv"


CSV file extracted


/usr4/ugrad/en/ml-herbarium/env/lib64/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,14,16,17,19,21,24,25,26,32,33,34,36,37,38,39,40,41,43,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Print Pandas Column Names

In [218]:
colnames = []
for col in df.columns:
    colnames.append(col)
print(colnames)


['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'infraspecificEpithet', 'taxonRank', 'scientificName', 'verbatimScientificName', 'verbatimScientificNameAuthorship', 'countryCode', 'locality', 'stateProvince', 'occurrenceStatus', 'individualCount', 'publishingOrgKey', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation', 'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day', 'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord', 'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber', 'identifiedBy', 'dateIdentified', 'license', 'rightsHolder', 'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted', 'mediaType', 'issue']


## Get Images

### Export GBIF Ocurrence IDs

In [434]:
data = {}

NUMBER_TO_SKIP = math.floor(df.shape[0] / (df.shape[0] * PERCENT_TO_SCRAPE))
NUMBER_TO_SCRAPE = math.ceil(df.shape[0] / NUMBER_TO_SKIP)
print(str(NUMBER_TO_SCRAPE) + " IDs will be scraped.")
for i in range(1, df.shape[0], NUMBER_TO_SKIP):
    if DATASET_TYPE == "dwca":
        id = df.at[i, "id"]
    elif DATASET_TYPE == "csv":
        id = df.at[i, "gbifID"]
    data[i] = {"id": str(id)}
print("Successfully scraped " + str(len(data)) + " IDs.")


1198 IDs will be scraped.
Successfully scraped 1198 IDs.


### Fetch Image URLs and Specimen Data from GBIF API

In [435]:
print("Data will be fetched for", len(data), "occurrences.")


def scrape_occurrence(key):
    rq = requests.get("https://api.gbif.org/v1/occurrence/" + str(data[key]["id"]))
    return_dict = {}
    return_dict[key] = {}
    return_dict[key]["img_url"] = json.loads(rq.content)["media"][0]["identifier"]
    return_dict[key]["img_type"] = json.loads(rq.content)["media"][0]["format"]
    return_dict[key]["country"] = json.loads(rq.content)["country"]
    return_dict[key]["genus"] = json.loads(rq.content)["genus"]
    return_dict[key]["species"] = json.loads(rq.content)["species"]
    return return_dict

print("Starting multiprocessing...")
pool = mp.Pool(NUM_CORES)
print("Fetching data...")
for item in tqdm(pool.imap(scrape_occurrence, data), total=len(data)):
    data.update(item)
pool.close()
pool.join()

print("\nSuccessfully fetched data for", len(data), "occurrences.")


Data will be fetched for 1198 occurrences.
Starting multiprocessing...
Fetching data...


100%|██████████| 1198/1198 [00:16<00:00, 73.99it/s]



Successfully fetched data for 1198 occurrences.


### Download Images

In [427]:
def download_images(key):
    img = requests.get(data[key]["img_url"], stream=True)
    with open(
        OUTPUT_PATH + str(key) + "." + data[1]["img_type"].split("/", 1)[1], "wb"
    ) as f:
        shutil.copyfileobj(img.raw, f)


if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

print("Starting multiprocessing...")
pool = mp.Pool(NUM_CORES)
print("Downloading images...")
for _ in tqdm(pool.imap(download_images, data), total=len(data)):
    pass
pool.close()


Starting multiprocessing...


100%|██████████| 1198/1198 [00:22<00:00, 54.02it/s]


### Export Geograpy Data

In [428]:
with open(OUTPUT_PATH + "countries.txt", "w") as f:
    for idx in data:
        f.write(data[idx]["country"] + "\n")
print("\nSuccessfully wrote countries to file.")



Successfully wrote countries to file.


### Export Taxon Data

In [429]:
with open(OUTPUT_PATH + "taxon.txt", "w") as f:
    for idx in data:
        f.write(data[idx]["genus"] + " " + data[idx]["species"] + "\n")
print("\nSuccessfully wrote taxon to file.")



Successfully wrote taxon to file.
